# Introduction

In this notebook, we process and prepare the UK Financial Sanctions List for comparison against bank customer records, potentially aiding in th compliance of regulatory requirements. The dataset must be extracted, cleaned, and structured to ensure accurate and thorough matching. Below is an outline of the key steps involved in this process:

- **Initial Data Exploration:** Exploring the dataset to get a sense of its size, shape, contents and any potential issues
- **Data Reshaping:** Modifying the structure of the data by deleting unnecessary columns, merging columns, and grouping rows
- **Data Repurposing:** Repurposing information for different columns (addressing some missing values)
- **Data Cleaning & Formatting:** Ensuring consistent formatting, fixing typos, removing invalid entries etc 

# Dataset Overview 

This dataset was sourced from the UK Financial Sanctions List provided by the Office of Financial Sanctions Implementation (OFSI). It contains information on individuals, entities, and ships and for the .txt, .csv and .xlsx formats has separate records for each permutation of data involving "Name," "Date of Birth," "Address," and "Regime." Each target is assigned a unique Group ID code, which is shared by all related records, but there is no unique code for individual records. For more detailed information on the dataset format, please refer to the [updated format guide for the Consolidated List](https://www.gov.uk/government/publications/financial-sanctions-consolidated-list-of-targets/consolidated-list-of-targets)

In addition the dataset contains detailed attributes about the targets, with all columns falling under one of the following categories:

* Name
* Address
* Personal Details (if applicable): Titles, aliases, nationalities, date of birth, etc.
* Identification Information: Passport numbers, identity cards, etc.
* Sanctions Information: Group ID, sanctions regime, relevant dates, etc.
* Other Information: A text column providing specific details on the sanctions, etc.


# Setup & Data Import

In [1]:
#import necessary libraries

import pandas as pd  #data processing
import numpy as np  #linear algebra
import nltk  #natural language processing
from nltk.tokenize import word_tokenize  #tokenization
import re  #regex 
from datetime import datetime  #dates format


#set display options for better readability 

pd.set_option('display.max_columns', None)  #display all columns 

In [2]:
#import csv file and convert it to dataframe

file_path='EDIT_THIS.csv'  #replace with respective file path 
df=pd.read_csv(file_path,skiprows=1)  #skip first row as it contains unecessary information 


# Initial Data Exploration

To start with, we take a look at the dataframe and its dimensions.

In [3]:
#replace missing or invaild with NaN for consistency

df.replace(["", "N/A", "NULL", "None", "na", "null", "NaN", "undefined", "unknown", "Not Available", "?", "[]"], np.nan, inplace=True)


In [4]:
#check first five dataframe entries

df.head()

,Name 6,Name 1,Name 2,Name 3,Name 4,Name 5,Title,Name Non-Latin Script,Non-Latin Script Type,Non-Latin Script Language,DOB,Town of Birth,Country of Birth,Nationality,Passport Number,Passport Details,National Identification Number,National Identification Details,Position,Address 1,Address 2,Address 3,Address 4,Address 5,Address 6,Post/Zip Code,Country,Other Information,Group Type,Alias Type,Alias Quality,Regime,Listed On,UK Sanctions List Date Designated,Last Updated,Group ID
0,MITHOO,Mian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pakistan,Pakistan,NaN,NaN,NaN,NaN,Cleric (“Pir”) of Bharchundi Sharif Shrine,Hafizabad Taluka Daharki,District Ghotki,NaN,NaN,NaN,NaN,NaN,Pakistan,(UK Sanctions List Ref):GHR0086. (UK Statement...,Individual,Primary name variation,NaN,Global Human Rights,09/12/2022,09/12/2022,09/12/2022,15672
1,MITHU,Mian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pakistan,Pakistan,NaN,NaN,NaN,NaN,Cleric (“Pir”) of Bharchundi Sharif Shrine,Hafizabad Taluka Daharki,District Ghotki,NaN,NaN,NaN,NaN,NaN,Pakistan,(UK Sanctions List Ref):GHR0086. (UK Statement...,Individual,Primary name variation,NaN,Global Human Rights,09/12/2022,09/12/2022,09/12/2022,15672
2,MITTO,Mian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pakistan,Pakistan,NaN,NaN,NaN,NaN,Cleric (“Pir”) of Bharchundi Sharif Shrine,Hafizabad Taluka Daharki,District Ghotki,NaN,NaN,NaN,NaN,NaN,Pakistan,(UK Sanctions List Ref):GHR0086. (UK Statement...,Individual,Primary name variation,NaN,Global Human Rights,09/12/2022,09/12/2022,09/12/2022,15672
3,MITTU,Mian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pakistan,Pakistan,NaN,NaN,NaN,NaN,Cleric (“Pir”) of Bharchundi Sharif Shrine,Hafizabad Taluka Daharki,District Ghotki,NaN,NaN,NaN,NaN,NaN,Pakistan,(UK Sanctions List Ref):GHR0086. (UK Statement...,Individual,Primary name variation,NaN,Global Human Rights,09/12/2022,09/12/2022,09/12/2022,15672
4,ZADACHIN,Andrei,Andreevich,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/08/1990,NaN,Russia,Russia,NaN,NaN,NaN,NaN,(1) Investigator for Particularly Important Ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(UK Sanctions List Ref):RUS1831. Financial san...,Individual,Primary name variation,NaN,Russia,21/04/2023,21/04/2023,21/04/2023,15890


In [5]:
#check dataframe size 

print("No. of rows:", df.shape[0])
print("No. of columns:", df.shape[1])

No. of rows: 18765
No. of columns: 36


Let's verify that there are no missing categories in the **Group Type** column before proceeding with further analysis.


In [6]:
#inspect 'Group Type' column

print("Categories:", df["Group Type"].unique().tolist())  #list unique values 
print("Missing values?",df["Group Type"].isna().any())  #check for missing values

Categories: ['Individual', 'Entity', 'Ship']
Missing values? False


Considering that different categories appear to have different attributes, we can analyse them independently and check the number of missing values.

In [7]:
#create new dataframes to explore data for different groups independently

df_ship = df[df["Group Type"] == "Ship"]
df_entity = df[df["Group Type"] == "Entity"]
df_individual = df[df["Group Type"] == "Individual"] 

#define function to count Nan values for each column and group type

def nan_count_table(df, name):
    nan_count=df.isna().sum()  #count number of Nan values in each series 
    nan_count_df= nan_count.reset_index()  #turn series into dataframe 
    nan_count_df.columns = ['Column', name]  #rename columns 
    return nan_count_df

#apply function to all 3 dataframes

nan_count_ship = nan_count_table(df_ship, 'Ship')
nan_count_entity = nan_count_table(df_entity, 'Entity')
nan_count_individual = nan_count_table(df_individual, 'Individual')

#merge 3 dataframes

raw_nan_count = nan_count_ship.merge(nan_count_entity, on='Column').merge(nan_count_individual, on='Column')


raw_nan_count


,Column,Ship,Entity,Individual
0,Name 6,0,0,0
1,Name 1,81,5018,1107
2,Name 2,81,5018,6420
3,Name 3,81,5018,12568
4,Name 4,81,5018,13413
5,Name 5,81,5018,13631
6,Title,81,5018,12613
7,Name Non-Latin Script,81,4342,10361
8,Non-Latin Script Type,81,4643,12691
9,Non-Latin Script Language,81,4619,12576


Since there are might be multiple entries for each **Group ID**, we can compare the total number of entries for each **Group Type** with the number of unique values to get an estimate of the actual number of targets.

In [8]:
#compare the number of entries with the number of unique ID for each category (individual, entity, ship) 

print("(Individual)")  #for individual type 
print("No. of entries:",df_individual.shape[0])
print("No. of unique IDs:",len(df_individual["Group ID"].unique().tolist()))

print("(Entity)")
print("No. of entries:",df_entity.shape[0])  #for entity type
print("No. of unique IDs:",len(df_entity["Group ID"].unique().tolist()))

print("(Ship)")
print("No. of entries:",df_ship.shape[0])  #for ship type
print("No. of unique IDs:",len(df_ship["Group ID"].unique().tolist()))

(Individual)
No. of entries: 13666
No. of unique IDs: 3643
(Entity)
No. of entries: 5018
No. of unique IDs: 1008
(Ship)
No. of entries: 81
No. of unique IDs: 15


We can also review **Other Information** for each category to understand the structure and content of the details provided.

In [9]:
#print the first cell under Other information for each category
#this is done for display purposes but in practice the unique values were inspected to identify patterns 

print("Entity")
print(df_entity["Other Information"].iloc[0])

print("Individual")
print(df_individual["Other Information"].iloc[0])

print("Ship")
print(df_ship["Other Information"].iloc[0])

Entity
(UK Sanctions List Ref):RUS2305. Financial sanctions imposed in addition to an asset freeze: Trust services. Date trust services sanctions imposed: 07/11/2024. (UK Statement of Reasons):Zapchasttrade LLP is an “involved person" under the Russia (Sanctions) (EU Exit) Regulations 2019 because it is or has been involved in obtaining a benefit from or supporting the Government of Russia by carrying on business in a sector of strategic significance to the Government of Russia, namely the Russian transport sector.  (Website):https://armtek.kz/ (Type of entity):Limited Liability Partnership (Business Reg No): (1) Kazakhstan BIN - 170440031562
Individual
(UK Sanctions List Ref):GHR0086. (UK Statement of Reasons):Mian Abdul HAQ is a cleric (“Pir”) at Bharchundi Sharif Shrine. HAQ is an involved person under the Global Human Rights Sanctions Regulations 2020 because he is or has been responsible for activity that abuses the right not to be subjected to torture or cruel, inhuman or degradi

For individuals and entities, the information in **Other Information** column was more varied. However, for ships, the data was more consistent, typically containing details about flags and ship owners. The cells for also included various identifiers (UK Sanctions List Ref, OFSI ID, IMO number, UN Ref).They appeared to correspond to the same Group ID, and therefore, were not preserved.

Some columns are related to non-Latin languages, which could potentially introduce complications. Therefore, it’s useful to list the languages involved to better understand the data.

In [10]:
#check non-latin languages 

df["Non-Latin Script Language"].unique().tolist()

[nan,
 'Persian',
 'Arabic',
 'Persian/Farsi',
 'Russian',
 'Hebrew',
 'Belarusian',
 'Chinese',
 'Ukrainian',
 'Cyrillic',
 'Bulgarian',
 'Spanish',
 'Turkish',
 'Bosnian',
 'Georgian',
 'Thai',
 'Greek',
 'Japanese',
 'Korean',
 'Burmese',
 'Ukraine',
 'Russia',
 'Kazakh',
 'Serbian',
 'Romanian']

# Data Reshaping

#### Remove columns

In [11]:
#remove columns with irrelevant or redundant information from dataframe

irrelevant_columns= ["Town of Birth", "Alias Type", "Alias Quality", "Listed On", "UK Sanctions List Date Designated"]
df.drop(columns=irrelevant_columns, inplace=True)

#### Merge columns 


All columns related to names and addresses were condensed into single columns for simplicity and match.

In [12]:
#define function that merges columns, returning a comma separated string

def merge_columns(row, columns):
    values=set(row[columns].dropna().astype(str))  #select non-Nan values from the specified columns in the row, 
                                                   #convert them to string and remove duplicates by converting to a set
    return ', '.join(filter(None, values))  #return a string with the unique values separated by a comma 


#list columns to join

name_columns=['Name 1', 'Name 2', 'Name 3', 'Name 4', 'Name 5', 'Name 6']  #name related columns
address_columns= ['Address 1', 'Address 2', 'Address 3', 'Address 4', 'Address 5', 'Address 6','Post/Zip Code','Country']  #address related columns

#create new columns with the merged columns

df['Name(s)']= df.apply(lambda row: merge_columns(row, name_columns), axis=1)  #apply function to name columns
df['Address(es)']= df.apply(lambda row: merge_columns(row, address_columns), axis=1)  #apply function to address columns

#remove name columns only for now

df.drop(columns=name_columns, inplace=True)


#### Merge rows

The objective is to ultimately compare the information with client records, so it is more effective to group the data by Group ID to ensure there is only one entry per target.

In [13]:
df.head()

,Title,Name Non-Latin Script,Non-Latin Script Type,Non-Latin Script Language,DOB,Country of Birth,Nationality,Passport Number,Passport Details,National Identification Number,National Identification Details,Position,Address 1,Address 2,Address 3,Address 4,Address 5,Address 6,Post/Zip Code,Country,Other Information,Group Type,Regime,Last Updated,Group ID,Name(s),Address(es)
0,NaN,NaN,NaN,NaN,NaN,Pakistan,Pakistan,NaN,NaN,NaN,NaN,Cleric (“Pir”) of Bharchundi Sharif Shrine,Hafizabad Taluka Daharki,District Ghotki,NaN,NaN,NaN,NaN,NaN,Pakistan,(UK Sanctions List Ref):GHR0086. (UK Statement...,Individual,Global Human Rights,09/12/2022,15672,"MITHOO, Mian","District Ghotki, Hafizabad Taluka Daharki, Pak..."
1,NaN,NaN,NaN,NaN,NaN,Pakistan,Pakistan,NaN,NaN,NaN,NaN,Cleric (“Pir”) of Bharchundi Sharif Shrine,Hafizabad Taluka Daharki,District Ghotki,NaN,NaN,NaN,NaN,NaN,Pakistan,(UK Sanctions List Ref):GHR0086. (UK Statement...,Individual,Global Human Rights,09/12/2022,15672,"MITHU, Mian","District Ghotki, Hafizabad Taluka Daharki, Pak..."
2,NaN,NaN,NaN,NaN,NaN,Pakistan,Pakistan,NaN,NaN,NaN,NaN,Cleric (“Pir”) of Bharchundi Sharif Shrine,Hafizabad Taluka Daharki,District Ghotki,NaN,NaN,NaN,NaN,NaN,Pakistan,(UK Sanctions List Ref):GHR0086. (UK Statement...,Individual,Global Human Rights,09/12/2022,15672,"MITTO, Mian","District Ghotki, Hafizabad Taluka Daharki, Pak..."
3,NaN,NaN,NaN,NaN,NaN,Pakistan,Pakistan,NaN,NaN,NaN,NaN,Cleric (“Pir”) of Bharchundi Sharif Shrine,Hafizabad Taluka Daharki,District Ghotki,NaN,NaN,NaN,NaN,NaN,Pakistan,(UK Sanctions List Ref):GHR0086. (UK Statement...,Individual,Global Human Rights,09/12/2022,15672,"MITTU, Mian","District Ghotki, Hafizabad Taluka Daharki, Pak..."
4,NaN,NaN,NaN,NaN,22/08/1990,Russia,Russia,NaN,NaN,NaN,NaN,(1) Investigator for Particularly Important Ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(UK Sanctions List Ref):RUS1831. Financial san...,Individual,Russia,21/04/2023,15890,"Andreevich, Andrei, ZADACHIN",


In [14]:
#group the data by ID 

df = df.groupby("Group ID").agg(
    
    #for columns other than 'Address(es)', convert each unique non-Nan value to string and join them with a comma 
    
    lambda x: ', '.join(x.dropna().astype(str).unique()) if x.name != 'Address(es)' 
    
    #for 'Address(es)', join unique non-NaN values using a newline to preserve full address formatting
    
    else '\n'.join(x.dropna().unique())
    
).reset_index()  #reset index to return dataframe


In [15]:
#display dataframe size to confirm the columns and rows were merged correctly 

df.shape

(4666, 27)

# Data Repurposing

### Other Information

As mentioned earlier, **Other Information** includes the names of the ship owners, which are more relevant to this analysis than the ship names themselves. Additionally, the country flag information can be used to fill in missing values for the associated countries.

In [16]:
#define function that that extracts country flag names and company names from 'Other Information'

def extract_info(text, pattern):
    match=re.search(pattern, text)  #searches fro regex pattern
    return match.group(1).strip() if match else None  #in case of a match retrieve the correct string and trailing and leading spaces

#define regex patterns

flag_pattern = r"\(Flag of ship\):([^\(\n]+)"  #extracts text after (Flag of ship):
name_pattern = r"Listed as asset of ([^\(\-]+)"  #extracts text after "Listed as asset of "


#apply flag pattern extraction function to 'Other Information' and store the value on 'Nationality'

df.loc[df["Group Type"]=="Ship", "Nationality"] = df.loc[df["Group Type"]=="Ship", "Other Information"].apply(lambda x: extract_info(x, flag_pattern))

#apply name pattern extraction function to 'Other Information' and replace the shp name in  'Name(s)

df.loc[df["Group Type"]=="Ship", "Name(s)"] = df.loc[df["Group Type"]=="Ship", "Other Information"].apply(lambda x: extract_info(x, name_pattern))


In [17]:
#print the new unique name values for ships

df[df["Group Type"]=="Ship"]["Name(s)"].unique().tolist()

['Chonmyong Shipping Co',
 'Korea Ansan Shipping Company',
 'Korea Myongdok Shipping Co',
 'Korea Samjong Shipping',
 'Korea Samma Shipping Co',
 'Korea Yujong Shipping Co Ltd',
 'Paekma Shipping Co',
 'Phyongchon Shipping & Marine',
 'Korea Achim Shipping Co',
 'Hapjanggang Shipping Corp',
 'Myohyang Shipping Co',
 'Phyongchon Shipping & Marine,',
 'Hapjanggang Shipping Corp,']

In [18]:
#print the new unique nationality values for ships

df[df["Group Type"]=="Ship"]["Nationality"].unique().tolist() 

['North Korea']

For entities, many entries contain phone numbers, email addresses, and website links, all of which could be crucial for matching purposes. For simplicity, the focus was placed on phone numbers.

In [19]:
#define function that that extracts phone numbers from text

def extract_phone_number(text):
    
    phone_pattern = r"\(Phone number\):([^\(\n]+)"  #extracts text after (Phone number:)
    matches = re.findall(phone_pattern, text)  #finds all phone numbers
    return "\n".join(matches).strip() if matches else None  #joins phone numbers using a newline separator to avoid conflict with commas
    
#applies extraction function to 'Other Infrormation' column and outputs result into new column 'Phone Number' 

df["Phone Number"] = df["Other Information"].apply(extract_phone_number)


In [20]:
#delete no longer necessary column

df = df.drop(columns=['Other Information'])


### Country

The Country column represents the address location. For entities, these values can be used to fill missing entries in Nationality, which will serve as the column listing associated countries for each target.

In [21]:
#check nationality unique values for entities

df[df["Group Type"]=="Entity"]["Nationality"].unique()

array([''], dtype=object)

In [22]:
#select the values from 'Country' and add them to 'Nationality' for entities

df.loc[df['Group Type'] == 'Entity', 'Nationality'] = df.loc[df['Group Type'] == 'Entity', 'Country']

In [23]:
#check updated nationality unique values for entities

df[df["Group Type"]=="Entity"]["Nationality"].unique()

array(['', 'Philippines', 'Afghanistan, Pakistan',
       'Bangladesh, Yemen, United States, Sudan', 'Algeria', 'Pakistan',
       'Afghanistan, Kuwait, United Arab Emirates, Pakistan', 'Lebanon',
       'United States', 'Tunisia', 'Morocco', 'Afghanistan',
       'Algeria, Mali, Mauritania, Morocco, Niger, Tunisia', 'Libya',
       'Pakistan, Afghanistan', 'United Kingdom', 'Kenya', 'Tanzania',
       'Iraq', 'Iraq, United Arab Emirates', 'Liechtenstein',
       'Switzerland', 'Bosnia and Herzegovina', 'Jordan', 'Netherlands',
       'Albania', 'Ethiopia', 'Bangladesh', 'Comoros',
       'Congo (Democratic Republic)', 'Uganda',
       'Congo (Democratic Republic), Rwanda', 'Iran', 'Zimbabwe',
       'North Korea', 'North Korea, China', 'Somalia',
       'United Arab Emirates', 'Mauritius', 'Malaysia', 'Iran, Venezuela',
       'Syria', 'Afghanistan, Pakistan, Russia', 'Turkey, Iran', 'Turkey',
       "Democratic People's Republic of Korea", 'Indonesia',
       'Afghanistan, United Ara

In [24]:
#delete no longer necessary address columns

df.drop(columns=address_columns, inplace=True)


In [25]:
#check new dataframe preview

df.head()

,Group ID,Title,Name Non-Latin Script,Non-Latin Script Type,Non-Latin Script Language,DOB,Country of Birth,Nationality,Passport Number,Passport Details,National Identification Number,National Identification Details,Position,Group Type,Regime,Last Updated,Name(s),Address(es),Phone Number
0,6894,,,,,"17/08/1958, 17/08/1957",(1) Indonesia (2) Indonesia,Indonesia,,,3603251708570001,,,Individual,ISIL (Da'esh) and Al-Qaida,08/02/2023,"Abu, ABDURRAHMAN, Jibril, ABDURRAHMAN, IQBAL, ...","Tangerang, Jalan Nakula, Indonesia, Komplek Wi...",None
1,6895,(1) Maulavi (2) Mullah,عبد الحی عظیم عبد القادر,,,00/00/1971,Afghanistan,Afghanistan,D 0001203,Issued in Afghanistan,,,"First Secretary, Taliban Consulate General, Qu...",Individual,Afghanistan,01/02/2021,"ABDUL QADER, Hazem, Abdul Hai, HAZEM, Abdul Hai","Kabul City, District Number 9, Puli Charkhi Ar...",None
2,6897,Haji,عبد المنان آغا,,,,,,,,,,,Individual,ISIL (Da'esh) and Al-Qaida,31/12/2020,"MANAN, AGHA, ABDUL, AL-MAN, Abd, Saiyid, MANAN...",,None
3,6899,,ثروت صالح شحاته,,,29/06/1960,Egypt,Egypt,,,,,,Individual,ISIL (Da'esh) and Al-Qaida,31/12/2020,"Salah, ABDALLAH, Tarwat, Salah, Tharwat, Shiha...",,None
4,6901,,,,,"15/04/1939, 00/00/1938",,Pakistan,,,,,,Individual,ISIL (Da'esh) and Al-Qaida,08/02/2023,"Majeed, ABDUL, Majeed, ABDUL, Chaudhry, ABDUL,...",,None


# Data Cleaning & Formatting

### Nationality

In [26]:
#inspect the unique values of the 'Nationality' column

df["Nationality"].unique()

array(['Indonesia', 'Afghanistan', '', 'Egypt', 'Pakistan', 'Iraq',
       'Tanzania', 'Libya', 'Mauritania', 'United Kingdom', 'Jordan',
       'Philippines', 'Afghanistan, Pakistan',
       '(1) Afghanistan (2) Pakistan',
       'Bangladesh, Yemen, United States, Sudan', 'Saudi Arabia',
       'Algeria', 'Afghanistan, Kuwait, United Arab Emirates, Pakistan',
       'Yemen', '(1) Jordan. (2) United States', 'Somalia',
       '(1) Algeria. (2) Palestine', 'Tunisia', 'Lebanon',
       '(1) Germany. (2) Morocco', 'Morocco', 'United States',
       'Algeria, Mali, Mauritania, Morocco, Niger, Tunisia',
       'Pakistan, Afghanistan', 'France', 'Malaysia', 'India', 'Kuwait',
       'Kenya', 'Iraq, United Arab Emirates', 'Liechtenstein',
       'Switzerland', 'Bosnia and Herzegovina', 'Netherlands', 'Albania',
       'Ethiopia', 'Bangladesh', 'Comoros', 'Congo (Democratic Republic)',
       'Uganda', 'Rwanda', 'Sudan', 'Belarus',
       'Congo (Democratic Republic), Rwanda', 'Iran', 'Zimbabw

At first glance, the uncleaned data shows the following issues:


* **Numbering:** (1) Afghanistan (2) Pakistan
* **Inconsistent delimiters:** 'Germany. Russia', 'Belarus, Russia'
* **Inconsistent names:** 'UK','United Kingdom'
* **Typos:** 'Belrus' 

In [27]:
#standardize country names using regex patterns

df["Nationality"]= df["Nationality"].str.replace(r"\(\d+\)\s*", ", ", regex=True)  #replace (1),(2) and (3) with comma
df["Nationality"]= df["Nationality"].str.replace(r"\.\s*", "", regex=True)  #remove extra dot
df["Nationality"]= df["Nationality"].str.replace(r"\s*,\s*", ", ", regex=True)  #remove spaces before comma
df["Nationality"]= df["Nationality"].str.replace(r"^[^a-zA-Z]+","",regex=True)  #remove everything before first character

#manually change inconssistent country names, fix typos and simplefy country name

manual_country_changes={"Congo (Democratic Republic)":"Democratic Republic of the Congo","Congo (Democratic Republic), Rwanda":
                        "Democratic Republic of the Congo, Rwanda","Democratic People's Republic of Korea": "North Korea",
                        "Central African Republic, Congo (Democratic Republic)":"Central African Republic, Democratic Republic of the Congo",
                        "Palestinian":"Palestine","Belgium, Congo (Democratic Republic)":"Belgium, Democratic Republic of the Congo",
                        "GermanyRussia":"Germany, Russia","Britain":"United Kingdom","Russian":"Russia","Repubic of Moldova, Israel":
                        "Moldova","UK, Russia":"United Kingdom, Russia","German":"Germany","United Republic of Tanzania":"Tanzania",
                        "Occupied Palestinian Territories":"Palestine","DPRK": "North Korea","The Gambia": "Gambia",
                        "Serbia, Bosnia-Herzegovina":"Serbia, Bosnia Herzegovina","Morocco, The Gambia":"Morocco, Gambia",
                        "Bosnia and Herzegovina":"Bosnia Herzegovina","Belrus": "Belarus"}

df["Nationality"]=df["Nationality"].replace(manual_country_changes)


In [28]:
#check result

df["Nationality"].unique()

array(['Indonesia', 'Afghanistan', '', 'Egypt', 'Pakistan', 'Iraq',
       'Tanzania', 'Libya', 'Mauritania', 'United Kingdom', 'Jordan',
       'Philippines', 'Afghanistan, Pakistan',
       'Bangladesh, Yemen, United States, Sudan', 'Saudi Arabia',
       'Algeria', 'Afghanistan, Kuwait, United Arab Emirates, Pakistan',
       'Yemen', 'Jordan, United States', 'Somalia', 'Algeria, Palestine',
       'Tunisia', 'Lebanon', 'Germany, Morocco', 'Morocco',
       'United States',
       'Algeria, Mali, Mauritania, Morocco, Niger, Tunisia',
       'Pakistan, Afghanistan', 'France', 'Malaysia', 'India', 'Kuwait',
       'Kenya', 'Iraq, United Arab Emirates', 'Liechtenstein',
       'Switzerland', 'Bosnia Herzegovina', 'Netherlands', 'Albania',
       'Ethiopia', 'Bangladesh', 'Comoros',
       'Democratic Republic of the Congo', 'Uganda', 'Rwanda', 'Sudan',
       'Belarus', 'Democratic Republic of the Congo, Rwanda', 'Iran',
       'Zimbabwe', 'Turkey', 'China', 'North Korea', 'Somalia, Sw

For reference, the following rules were used when cleaning the country names: 

- **No abreviations**:
  - 'United Kingdom' instead of 'UK' 
- **Simplified name version**:
  - 'North Korea' instead of 'Democratic People's Republic of Korea'
  - 'Bosnia Herzegovina' instead of 'Bosnia and Herzegovina'
- **'Republic of' Removed When Possible**:  
  - 'Moldova' instead of 'Republic of Moldova'  
  - 'Democratic Republic of the Congo' remained unchanged to avoid confusion with 'Republic of Congo'  
- **Hyphens Removed**:  
  - note that this might be problematic when distinguishing between:  
    - 'South Sudan' / 'Sudan'  
    - 'Guinea' / 'Guinea-Bissau' / 'Equatorial Guinea'  


In [29]:
#rename 'Nationality' 

df.rename(columns={"Nationality": "Countries"}, inplace=True)

### DOB

In [30]:
#inspect 'DOB'

df["DOB"].unique() 

array(['17/08/1958, 17/08/1957', '00/00/1971', '', ..., '06/07/1969',
       '15/01/1991', '12/03/1971'], dtype=object)

Upon inspection, the date format appeared to be consistent throughout the dataset in **dd/mm/yyyy** and separated by commas. 
However, it was converted to the international date format **yyyy-mm-dd** (ISO 8601), as it is more widely adopted.  


In [31]:
#convert a string of dates into a list of dates

df["DOB"]=df["DOB"].apply(lambda x: x.split(", ") if isinstance(x, str) else [])  

#define a function that filters out invalid dates and converts the valid ones to international format

def filter_and_format_dates(dob_list):
    valid_dates = []  #initialize list of valid dates
    #cleaned_dates = []  #initialize list of cleaned dates 
    for date in dob_list:  #loop through every date on the list
        if date.startswith("00/00/"):   #skip if the date only has the year infromation
            continue 
            
        date_match = re.match(r"^(\d{2})/(\d{2})/(\d{4})$", date)  #check if the date follows the expected dd/mm/yyyy format using regex
        
        if date_match:
            day, month, year = map(int, date_match.groups())  #if so, unpack day, month and year
            if 1<=day<=31 and 1<=month<=12:  #check if the days and months are within valid range 
                formatted_date = datetime(year, month, day).strftime('%Y-%m-%d')  #convert date frmat to internationsal using datetime pacakge
                valid_dates.append(formatted_date)  #append valid and formatted dates to valid_dates
                #cleaned_dates.append(date) 
        else:
            continue  # Skip invalid formats

    return valid_dates  #return valid dates 


df['DOB'] = df['DOB'].apply(lambda dob: filter_and_format_dates(dob))  #apply filter_and_format_dates function to 'DOB'

df['DOB'] = df['DOB'].apply(lambda dob_list: ', '.join(dob_list) if dob_list else '')  #converts the list back to a string separated by commas

In [32]:
#rename 'DOB' column

df.rename(columns={"DOB": "Date of Birth"}, inplace=True)

In [33]:
#check formated values

df["Date of Birth"].unique()

array(['1958-08-17, 1957-08-17', '', '1960-06-29', ..., '1969-07-06',
       '1991-01-15', '1971-03-12'], dtype=object)

### Last updated

In [34]:
#Inspect 'Last Updated'

df["Last Updated"].unique()

array(['08/02/2023', '01/02/2021', '31/12/2020', '11/02/2022',
       '12/01/2022', '11/01/2024', '08/01/2024', '11/03/2022',
       '21/01/2021', '16/06/2022', '14/04/2022', '21/01/2020',
       '22/09/2023', '16/11/2023', '18/02/2021', '16/02/2022',
       '25/07/2023', '01/04/2021', '15/03/2022', '14/06/2022',
       '19/04/2022', '20/01/2021', '17/05/2024', '07/03/2023',
       '19/01/2021', '07/04/2021', '18/03/2022', '05/06/2023',
       '04/03/2022', '14/12/2023', '10/02/2022', '02/08/2022',
       '03/07/2023', '03/03/2022', '05/12/2024', '08/03/2022',
       '13/05/2022', '16/11/2020', '13/05/2021', '30/11/2023',
       '30/04/2021', '20/07/2022', '14/02/2022', '19/05/2022',
       '14/01/2023', '20/10/2023', '21/08/2023', '10/03/2022',
       '21/03/2023', '02/12/2021', '15/01/2024', '24/03/2021',
       '23/02/2024', '08/01/2021', '17/02/2022', '25/03/2021',
       '17/01/2022', '16/04/2024', '07/07/2022', '16/09/2022',
       '07/07/2023', '20/09/2024', '11/03/2024', '28/01

The format of these dates appears to be similar to the orginal format in 'DOB'. To ensure consistency, the dates will be converted in the same way.

In [35]:
#convert a string of dates into a list of dates

df["Last Updated"]=df["Last Updated"].apply(lambda date: date.split(", ") if isinstance(date, str) else []) 

#apply filter_and_format_dates function to 'Last Updated'

df['Last Updated']= df['Last Updated'].apply(lambda date: filter_and_format_dates(date))  

#convert list back to a string separted by commas 

df['Last Updated']=df['Last Updated'].apply(lambda date: ', '.join(date) if date else '')


### Name(s)

In [36]:
#inspect the name format for individuals

df[df["Group Type"]=="Individual"]["Name(s)"].unique()  #extract unique values

array(['Abu, ABDURRAHMAN, Jibril, ABDURRAHMAN, IQBAL, MOHAMAD, A Rahman, IQBAL, Mohamad, Abdul Rahman, IQBAL, Mohamad, IQBAL, Rahman, Mohamad, Fihiruddin, MUQTI, MUQTI, Fikiruddin',
       'ABDUL QADER, Hazem, Abdul Hai, HAZEM, Abdul Hai',
       "MANAN, AGHA, ABDUL, AL-MAN, Abd, Saiyid, MANAN, Abdul, Man'am, Abdul, SAIYID",
       ..., 'RI SONG JIN, RI SONG-JIN',
       'Natalia, Aleksandrovna, TYURINA, Aleksandrovna, Natalya, TYURINA',
       'Vasilievich, OREKHOV, Aleksandr, Vasilyevich, OREKHOV, Aleksandr'],
      dtype=object)

In [37]:
#inspect the name format for entities 

df[df["Group Type"]=="Entity"]["Name(s)"].unique()  #extract the unique values

array(['ABU NIDAL ORGANISATION, ANO',
       'ABU SAYYAF GROUP, AL HARAKAT AL ISLAMIYYA',
       'AFGHAN SUPPORT COMMITTEE (ASC), AHYA UL TURAS, JAMIAT AYAT-UR-RHAS AL ISLAMIAC, JAMIAT IHYA UL TURATH AL ISLAMIA, LAJNAT UL MASA EIDATUL AFGHANIA',
       ...,
       'LIMITED LIABILITY COMPANY IMPEX-DON, LLC IMPEX-DON, OOO "IMPEKS-DON"',
       'POLY TECHNOLOGIES INCORPORATED, POLY TECHNOLOGIES, INC, POLYTECH, PTI',
       'OAO KEREMET BANK, OJCS KEREMET BANK, OJSC KEREMET BANK, OJSC ROSINBANK, OPEN JOINT STOCK COMPANY KEREMET BANK, OTKRYTOE AKTSIONERNOE OBSCHESTVO KEREMET BANK'],
      dtype=object)

In [38]:
#inspect the name format for ships (owners)

df[df["Group Type"] == "Ship"]["Name(s)"].unique().tolist()  #extract and list the unique values

['Chonmyong Shipping Co',
 'Korea Ansan Shipping Company',
 'Korea Myongdok Shipping Co',
 'Korea Samjong Shipping',
 'Korea Samma Shipping Co',
 'Korea Yujong Shipping Co Ltd',
 'Paekma Shipping Co',
 'Phyongchon Shipping & Marine',
 'Korea Achim Shipping Co',
 'Hapjanggang Shipping Corp',
 'Myohyang Shipping Co',
 'Phyongchon Shipping & Marine,',
 'Hapjanggang Shipping Corp,']

The names of the targets were inspected independently according to their **Group Type** to identify their particluar structure. The common rules applied in all three cases were: 

1. **Extra spaces removed**  
2. **Parentheses removed**  
3. **Hyphens preserved**  
4. **Apostrophes preserved**  
5. **Duplicates removed**


For individuals, only the first letter of each word was capitalized. Since there were multiple variations of first, middle, and surnames in different language structures, each word was separated by a comma. The idea was to preserve the totality of the data, allowing for different combinations of names to be tested for match, similarly to tokens in tokenization, thus guaranteeing a match.

For entities (including the companies owning the ships), the entire name was capitalized. Acronyms were preserved, and abbreviations were kept in their short format.


In [39]:
#define function to clean names for individuals

def clean_names_individual(name_string):

    name_string = re.sub(r'\(|\)', '', name_string)  #remove parentheses
    name_string= re.sub(r'\.', '', name_string)  #remove dots   
    name_list=re.split(r'[\s,]+', name_string) #split the string wherever there are spaces or commas to create a list of substrings
    cleaned_names=[name.strip().title() for name in name_list if name.strip()] #remove extra spaces and capitalize first letter
    cleaned_names= sorted(set(cleaned_names))  #remove duplicates by converting to a set and back to a list 
 
    return ', '.join(cleaned_names)  #return a string separated by commas


#apply clean_names_individual to individual 'Name(s)'

df.loc[df["Group Type"] == "Individual", "Name(s)"] = df.loc[df["Group Type"] == "Individual", "Name(s)"].apply(clean_names_individual)



In [40]:
#define function to clean names for entities

def clean_names_entity(name):
    
    name= name.upper()  #convert name to uppercase
    name= re.sub(r'\(|\)', '', name)  #remove parentheses
    name= re.sub(r'\.', '', name)  #remove dots   
    name= name.replace("&", "AND")  #replace & with 'AND'
    
    name= re.sub(r'\bLIMITED\b|\bLTD\b', 'LTD', name)  # Replace 'LIMITED' with 'LTD'
    name= re.sub(r'\bCORPORATION\b|\bCO\b', 'CO', name)  # Replace 'CORPORATION' or 'CO' with 'CO'
    name= re.sub(r'\bINCORPORATED\b|\bINC\b', 'INC', name)  # Replace 'INCORPORATED' with 'INC'
    name= re.sub(r'\bLLC\b', 'LLC', name)  # Keep LLC as is

    #eliminate redundant variations of 'SHIPPING CO'
    
    name= re.sub(r'\bSHIPPING CO\b|\bSHIPPING COMPANY\b|\bSHIPPING CORP\b', 'SHIPPING CO', name) 
    
    name= re.sub(r'\s*,\s*', ', ', name).strip(', ')  #remove extra commas or spaces
    
    
    return name
    

#apply clean_names_entity function to Name(s) in both entity and ship

df.loc[df["Group Type"]=="Entity", "Name(s)"]= df.loc[df["Group Type"]== "Entity", "Name(s)"].apply(clean_names_entity)
df.loc[df["Group Type"]=="Ship", "Name(s)"]= df.loc[df["Group Type"]== "Ship", "Name(s)"].apply(clean_names_entity)



In [41]:
#check cleaned names 

df[df["Group Type"] == "Individual"]["Name(s)"].unique()

array(['A, Abdul, Abdurrahman, Abu, Fihiruddin, Fikiruddin, Iqbal, Jibril, Mohamad, Muqti, Rahman',
       'Abdul, Hai, Hazem, Qader',
       "Abd, Abdul, Agha, Al-Man, Man'Am, Manan, Saiyid", ...,
       'Jin, Ri, Song, Song-Jin',
       'Aleksandrovna, Natalia, Natalya, Tyurina',
       'Aleksandr, Orekhov, Vasilievich, Vasilyevich'], dtype=object)

In [42]:
#check cleaned names

df[df["Group Type"] == "Entity"]["Name(s)"].unique()

array(['ABU NIDAL ORGANISATION, ANO',
       'ABU SAYYAF GROUP, AL HARAKAT AL ISLAMIYYA',
       'AFGHAN SUPPORT COMMITTEE ASC, AHYA UL TURAS, JAMIAT AYAT-UR-RHAS AL ISLAMIAC, JAMIAT IHYA UL TURATH AL ISLAMIA, LAJNAT UL MASA EIDATUL AFGHANIA',
       ...,
       'LTD LIABILITY COMPANY IMPEX-DON, LLC IMPEX-DON, OOO "IMPEKS-DON"',
       'POLY TECHNOLOGIES INC, POLY TECHNOLOGIES, INC, POLYTECH, PTI',
       'OAO KEREMET BANK, OJCS KEREMET BANK, OJSC KEREMET BANK, OJSC ROSINBANK, OPEN JOINT STOCK COMPANY KEREMET BANK, OTKRYTOE AKTSIONERNOE OBSCHESTVO KEREMET BANK'],
      dtype=object)

In [43]:
#check cleaned names

df[df["Group Type"] == "Ship"]["Name(s)"].unique().tolist()

['CHONMYONG SHIPPING CO',
 'KOREA ANSAN SHIPPING CO',
 'KOREA MYONGDOK SHIPPING CO',
 'KOREA SAMJONG SHIPPING',
 'KOREA SAMMA SHIPPING CO',
 'KOREA YUJONG SHIPPING CO LTD',
 'PAEKMA SHIPPING CO',
 'PHYONGCHON SHIPPING AND MARINE',
 'KOREA ACHIM SHIPPING CO',
 'HAPJANGGANG SHIPPING CO',
 'MYOHYANG SHIPPING CO']

## Final Dataset

In [44]:
#remove relevant unaddressed columns 

df = df.drop(columns=['Non-Latin Script Type', 'Non-Latin Script Language', 'Country of Birth', 'Passport Number', 
                      'Passport Details', 'National Identification Number', 'National Identification Details','Title','Position'])

#reorder columns to display Sanctions Related columns first, then target specific columns

df = df[['Group ID', 'Group Type', 'Last Updated', 'Regime', 'Name(s)', 'Date of Birth', 'Countries', 'Address(es)', 'Phone Number',"Name Non-Latin Script"]]


df.head()  #check five first entries

,Group ID,Group Type,Last Updated,Regime,Name(s),Date of Birth,Countries,Address(es),Phone Number,Name Non-Latin Script
0,6894,Individual,2023-02-08,ISIL (Da'esh) and Al-Qaida,"A, Abdul, Abdurrahman, Abu, Fihiruddin, Fikiru...","1958-08-17, 1957-08-17",Indonesia,"Tangerang, Jalan Nakula, Indonesia, Komplek Wi...",None,
1,6895,Individual,2021-02-01,Afghanistan,"Abdul, Hai, Hazem, Qader",,Afghanistan,"Kabul City, District Number 9, Puli Charkhi Ar...",None,عبد الحی عظیم عبد القادر
2,6897,Individual,2020-12-31,ISIL (Da'esh) and Al-Qaida,"Abd, Abdul, Agha, Al-Man, Man'Am, Manan, Saiyid",,,,None,عبد المنان آغا
3,6899,Individual,2020-12-31,ISIL (Da'esh) and Al-Qaida,"Abdallah, Ali, Salah, Shahata, Shihata, Tarwat...",1960-06-29,Egypt,,None,ثروت صالح شحاته
4,6901,Individual,2023-02-08,ISIL (Da'esh) and Al-Qaida,"Abdul, Chaudhry, Majeed, Majid",1939-04-15,Pakistan,,None,


In [52]:
#convert dataframe to csv file 

f.to_csv('output_data.csv', index=False)

ERROR! Session/line number was not unique in database. History logging moved to new session 3


# Important Considerations

### Uncleaned Data

The following columns were not cleaned:
- 'Regime'
- 'Address(es)' 
- 'Phone Number'
- 'Name Non-Latin Script'

### Deleted Data

The following columns were deleted but could be useful for matching targets:
- 'Passport Details'
- 'National Identification Number'
- 'National Identification Details'

The following columns were deleted but could be used to manually search fo missing values:

- 'Title'
- 'Position'
- 'Country of Birth'

Title' and 'Position' could be used to identify an individual and help retrieve basic information such as 'Date of Birth' from the internet. Additionally, the 'Country of Birth' could be useful for countries that offer birthright citizenship, as it may help fill in missing values in the 'Nationality' column.
